In [1]:
import os
os.chdir("../")

In [2]:
!pwd

/Users/ngkuissi/Dev/projects/Medical-Chatbot


In [15]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

In [13]:
load_dotenv()

True

In [4]:
def load_pdf_file(path):
    loader = DirectoryLoader(
        path, glob="medical_book.pdf", loader_cls=PyPDFLoader
    )
    return loader.load()

In [5]:
extracted_data = load_pdf_file("data/")

In [6]:
extracted_data[300].page_content

'diaphragm is effective only if used during every episode\nof sexual intercourse. The same is true for condoms and\nthe cervical cap. Some methods are automatically work-\ning every day, no matter what. These methods include\nDepo Provera, Norplant, the IUD, and tubal sterilization.\nThere are many different ways to use birth control.\nThey can be divided into several groups:\n• By mouth (oral)—Birth control pills must be taken by\nmouth every day.\n• Injected—Depo Provera is a hormonal medication that\nis given by injection every three months.\n• Implanted—Norplant is a long-acting hormonal form\nof birth control that is implanted under the skin of the\nupper arm.\n• Vaginal—Spermicides and barrier methods work in the\nvagina.\n• Intra-uterine—The IUD is inserted into the uterus.\n• Surgical—Tubal sterilization is a form of surgery. A\ndoctor must perform the procedure in a hospital or sur-\ngical clinic. Many women need general anesthesia.\nThe methods of birth control differ from ea

In [7]:
def split_text(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = split_text(extracted_data)
print("Length of chunks:", len(text_chunks))

Length of chunks: 7023


In [9]:
def downlaod_hugging_face_embeddings():
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

In [10]:
embedding_model = downlaod_hugging_face_embeddings()

/Users/ngkuissi/miniforge3/envs/medical_chatbot/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
query_result = embedding_model.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [21]:
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
pc

In [24]:
index_name = "medical-bot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)